## Fase 3: Preparação dos Dados

### Importações de bibliotecas e carregamento dos dados


In [2]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Carregar dataset bruto
file_path = '../data/raw/desafio_indicium_imdb.csv'
df = pd.read_csv(file_path, index_col=0) # essa coluna é irrelevante pra análise pois é apenas um índice sequencial

# uma copia do dataset pra não alterar original
data = df.copy()

### Limpeza de Dados

In [4]:
# tratando valores ausentes
print(data.isnull().sum())

# Estratégias usadas:
"""
manter certificate como 'Unknown'
preencher meta score com mediana
em Gross é melhor remover os filmes sem informação, pois é uma métrica crítica
"""

data['Certificate'] = data['Certificate'].fillna("Unknown")
data['Meta_score'] = data['Meta_score'].fillna(data['Meta_score'].median())
data = data.dropna(subset=['Gross'])


Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64


### Meta_score → versão com média

In [5]:
# Criar versão alternativa do dataset com Meta_score preenchido pela média
data_mean = df.copy()
data_mean['Certificate'] = data_mean['Certificate'].fillna("Unknown")
data_mean['Meta_score'] = data_mean['Meta_score'].fillna(data_mean['Meta_score'].mean())
data_mean = data_mean.dropna(subset=['Gross'])

print("Valores ausentes (versão com média):")
print(data_mean.isnull().sum())


Valores ausentes (versão com média):
Series_Title     0
Released_Year    0
Certificate      0
Runtime          0
Genre            0
IMDB_Rating      0
Overview         0
Meta_score       0
Director         0
Star1            0
Star2            0
Star3            0
Star4            0
No_of_Votes      0
Gross            0
dtype: int64


In [6]:
# Correção dos tipos de dados

# Runtime: extrair apenas os números e converter
data['Runtime'] = (
    data['Runtime']
    .astype(str)                       # garante string
    .str.extract(r'(\d+)')[0]          # pega só os números
    .astype(float)
    .astype("Int64")                   # int compatível com NaN
)

# Gross: remover vírgulas e converter para float
data['Gross'] = (
    data['Gross']
    .astype(str)
    .str.replace(",", "", regex=False)
    .astype(float)
)

# Released_Year: garantir que seja numérico
data['Released_Year'] = pd.to_numeric(data['Released_Year'], errors='coerce')

In [7]:
# remoção da coluna 'Unnamed: 0'
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

### Engenharia de Variáveis

In [8]:
# Gênero Principal
data['Main_Genre'] = data['Genre'].apply(lambda x: x.split(",")[0])

# Dummies para gêneros principais
genre_dummies = pd.get_dummies(data['Main_Genre'], prefix="Genre")
data = pd.concat([data, genre_dummies], axis=1)

In [9]:
# Popularidade de Diretor (Top 20)

director_counts = data['Director'].value_counts()
top_directors = director_counts.index[:20]
data['Director_clean'] = data['Director'].apply(lambda x: x if x in top_directors else "Other")

In [10]:
# Popularidade de Atores (Top 20)
actors = pd.concat([data['Star1'], data['Star2'], data['Star3'], data['Star4']])
actor_counts = actors.value_counts()
top_actors = actor_counts.index[:20]
for col in ['Star1', 'Star2', 'Star3', 'Star4']:
    data[col] = data[col].apply(lambda x: x if x in top_actors else "Other")

In [11]:
# Idade do Filme
current_year = 2025
data['Movie_Age'] = current_year - data['Released_Year']

### Preparação de Texto (Overview)

#### Redução de dimensionalidade no TF-IDF

In [12]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

data['Overview_clean'] = data['Overview'].apply(clean_text)

In [13]:
# Vetorização TF-IDF
tfidf = TfidfVectorizer(max_features=500)
overview_tfidf = tfidf.fit_transform(data['Overview_clean'])

# transformação para dataframe
overview_tfidf_df = pd.DataFrame(overview_tfidf.toarray(), columns=tfidf.get_feature_names_out())

### TF-IDF original + SVD (versões separadas)

In [14]:
# Dataset com TF-IDF original - o de 500 features
data_tfidf = pd.concat([data.reset_index(drop=True), overview_tfidf_df.reset_index(drop=True)], axis=1)

# Dataset com SVD reduzido (100 features)
svd = TruncatedSVD(n_components=100, random_state=42)
overview_svd = svd.fit_transform(overview_tfidf)
overview_svd_df = pd.DataFrame(overview_svd, columns=[f"svd_{i}" for i in range(1, 101)])
data_svd = pd.concat([data.reset_index(drop=True), overview_svd_df.reset_index(drop=True)], axis=1)

print("Dimensão TF-IDF original:", overview_tfidf_df.shape[1])
print("Dimensão reduzida com SVD:", overview_svd_df.shape[1])

Dimensão TF-IDF original: 500
Dimensão reduzida com SVD: 100


### Multi-label One-Hot Encoding para coluna Genre

In [15]:
genre_dummies_full = data['Genre'].str.get_dummies(sep=",")

# Remover espaços extras nos nomes das colunas
genre_dummies_full.columns = genre_dummies_full.columns.str.strip()

# Concatenar ao dataset base
data = pd.concat([data, genre_dummies_full], axis=1)

print("Total de colunas de gênero criadas:", genre_dummies_full.shape[1])


Total de colunas de gênero criadas: 33


### Escalonamento de variáveis numéricas

In [16]:
numeric_cols = ['Gross', 'No_of_Votes', 'Runtime', 'Movie_Age', 'Meta_score']

scaler = StandardScaler()
data_scaled = data.copy()
data_scaled[numeric_cols] = scaler.fit_transform(data[numeric_cols])

print("Exemplo de valores escalonados:")
print(data_scaled[numeric_cols].head())


Exemplo de valores escalonados:
      Gross  No_of_Votes   Runtime  Movie_Age  Meta_score
1  0.609468     3.887505  1.852863   1.036420    1.885361
2  4.253417     5.917729  1.016329  -0.665887    0.538240
3 -0.098254     2.429453  2.834881   0.941847    1.043410
4 -0.580661     1.120972 -1.020448   1.745714    1.548581
5  2.822667     3.954076  2.798510  -0.429455    1.380190


### Ordem das features e salvar features.json

In [17]:
import json

# colunas duplicadas - remover
data = data.loc[:, ~data.columns.duplicated()]
data_tfidf = data_tfidf.loc[:, ~data_tfidf.columns.duplicated()]
data_svd = data_svd.loc[:, ~data_svd.columns.duplicated()]
data_scaled = data_scaled.loc[:, ~data_scaled.columns.duplicated()]

# Reindexar em ordem alfabética
data = data.reindex(sorted(data.columns), axis=1)
data_tfidf = data_tfidf.reindex(sorted(data_tfidf.columns), axis=1)
data_svd = data_svd.reindex(sorted(data_svd.columns), axis=1)
data_scaled = data_scaled.reindex(sorted(data_scaled.columns), axis=1)

# Salvar lista de features
features = {
    "data": list(data.columns),
    "data_tfidf": list(data_tfidf.columns),
    "data_svd": list(data_svd.columns),
    "data_scaled": list(data_scaled.columns)
}

with open("../data/processed/features.json", "w") as f:
    json.dump(features, f, indent=4)

print("Lista de features salva em ../data/processed/features.json")

Lista de features salva em ../data/processed/features.json


### Salvar dataset processado

In [18]:
# Salvar datasets processados

output_path = '../data/processed/imdb_clean.csv'
data.to_csv(output_path, index=False)
print(f"Dataset base salvo em {output_path} (bom para árvores)")

output_path_scaled = '../data/processed/imdb_clean_scaled.csv'
data_scaled.to_csv(output_path_scaled, index=False)
print(f"Dataset escalonado salvo em {output_path_scaled} (bom para lineares/redes)")

output_path_tfidf = '../data/processed/imdb_clean_tfidf.csv'
data_tfidf.to_csv(output_path_tfidf, index=False)
print(f"Dataset TF-IDF salvo em {output_path_tfidf}")

output_path_svd = '../data/processed/imdb_clean_svd.csv'
data_svd.to_csv(output_path_svd, index=False)
print(f"Dataset SVD salvo em {output_path_svd}")

output_path_mean = '../data/processed/imdb_clean_mean.csv'
data_mean.to_csv(output_path_mean, index=False)
print(f"Dataset alternativo salvo em {output_path_mean} (Meta_score = média)")

Dataset base salvo em ../data/processed/imdb_clean.csv (bom para árvores)
Dataset escalonado salvo em ../data/processed/imdb_clean_scaled.csv (bom para lineares/redes)
Dataset TF-IDF salvo em ../data/processed/imdb_clean_tfidf.csv
Dataset SVD salvo em ../data/processed/imdb_clean_svd.csv
Dataset alternativo salvo em ../data/processed/imdb_clean_mean.csv (Meta_score = média)
